In [94]:
#ASSOCIATION ANALYSIS ON GENE DATA FOLLOWING APRIORI PRINCIPLE.
import numpy as np

def freq_itemsets(data, sup):
    itemsize = 1
    freq_item_count = 1;
    
    k_itemset_list = []
    freq_result = {}
    all_freq_items_map = {}
    all_freq_items_list = []
    while freq_item_count > 0:
        k_itemset_list, all_freq_items_map = generate_itemsets(k_itemset_list, data, itemsize,sup,all_freq_items_map) # generate item sets for given itemsize
        if itemsize > 1:     #rules cannot be generated for itemsets with length > 1
            all_freq_items_list = all_freq_items_list + k_itemset_list
        freq_item_count = len(k_itemset_list)
        if freq_item_count > 0:
            freq_result.update({itemsize : freq_item_count})
        itemsize += 1
    print(freq_result)
    return freq_result, all_freq_items_list, all_freq_items_map
    
#prune infreq ones that have less support count
def prune_infreq_sets(itemset, support_threshold, freq_item_count):
    for gene in list(itemset.keys()):
        if itemset.get(gene) >= support_threshold:
            freq_item_count.update({gene : itemset.get(gene)})
        else :
            del itemset[gene]
    return list(itemset),freq_item_count

def generate_itemsets(prev_itemset, dataset, itemsize,support_threshold,freq_item_count):
    if itemsize == 1:
        genes_set={}
        gc = 0
        for gene_form in dataset:
            for gene in gene_form:
                gc += 1
                if gene in genes_set:
                    genes_set.update({gene:genes_set.get(gene) +1})
                else: 
                    genes_set.update({gene:1})
        freq_genes_list, freq_item_count = prune_infreq_sets(genes_set, support_threshold,freq_item_count)
        return freq_genes_list, freq_item_count
    else:
        i=0
        freq_genes_list2 = []
        for gene_first in prev_itemset[i:]:
            for gene_second in prev_itemset[i+1:]:
                if itemsize == 2:
                    # Forming a tuple with gene1 and gene2--> combinations of length 2.
                    gene_perm_set = (gene_first, gene_second)
                    count = 0
                    #Increment count if the tuple if subset of a gene form.
                    for gene_form in dataset:
                        if set(gene_perm_set).issubset(gene_form):
                            count += 1
                    if count >= support_threshold:
                        freq_item_count.update({",".join(list(gene_perm_set)):count})
                        freq_genes_list2.append(list(gene_perm_set)) 
                else:
                    match = True
                    for g in range(itemsize - 2):
                        if gene_first[g] != gene_second[g]:
                            match = False
                            break
                    if match:
                        gene_k_set = []+(gene_first)
                        gene_k_set.append(gene_second[len(gene_second) - 1])
                        sup_cnt = 0
                        #Increment count if the tuple is subset of a gene form.
                        for gene_form in dataset:
                            if set(gene_k_set).issubset(gene_form):
                                sup_cnt += 1
                        if sup_cnt >= support_threshold:
                            freq_item_count.update({",".join(gene_k_set):sup_cnt})
                            freq_genes_list2.append(gene_k_set)
                    else: 
                        break
            i += 1
    return freq_genes_list2, freq_item_count


def generate_gene_rules(all_item_set,conf_percentage,freq_itemset_map):
    # Rule form : head ==> body
    head_list=[]
    body_list=[]
    for itemset in all_item_set:
        head=[]
        body=[]
        for i in range(len(itemset)):
            temp_itemset = itemset[:]
            hele=",".join(itemset[i:i+1])
            del temp_itemset[i]
            bele=",".join(temp_itemset)
            freq_set_str = ",".join(itemset)
            if (freq_itemset_map.get(freq_set_str)/(freq_itemset_map.get(hele)))*100 >= conf_percentage:
                if hele not in head: 
                    body.append(bele)
                    head.append(hele)
            if (freq_itemset_map.get(freq_set_str)/(freq_itemset_map.get(bele)))*100 >= conf_percentage:
                if hele not in body:
                    body.append(hele)
                    head.append(bele)
            if len(itemset)>2:
                for j in range(i+1, len(itemset)):
                    part1=[]   
                    temp_itemset = itemset[:]
                    g1 = itemset[i]
                    g2 = itemset[j]
                    part1.append(g1)
                    part1.append(g2)
                    temp_itemset.remove(g1)
                    temp_itemset.remove(g2)
                    part1=",".join(part1)
                    temp_itemset=",".join(temp_itemset)
                    if (freq_itemset_map.get(freq_set_str)/(freq_itemset_map.get(part1)))*100 >= conf_percentage:
                        if part1 not in head:
                            head.append(part1)
                            body.append(temp_itemset)
                    if (freq_itemset_map.get(freq_set_str)/(freq_itemset_map.get(temp_itemset)))*100 >= conf_percentage:
                        if part1 not in body:
                            body.append(part1)
                            head.append(temp_itemset)
        
        head_list = head_list + (head)
        body_list = body_list + (body)
    return head_list, body_list

    
def template1_results(arg1, arg2, arg3, head_list, body_list):
    genes = arg3.split(",")
    rule_cnt = 0
    rule_list = []
    rules_ind = []
    delete_rule = []
    if(arg1.lower() == 'rule'):
        for g in genes:
            if arg2.lower() == 'any':
                for i in range(0,len(head_list)):
                    if (g.lower() in head_list[i]) or (g.lower() in body_list[i]):
                        h = head_list[i]
                        b = body_list[i]
                        r = h +' --> '+b
                        rule_list.append(r)
                        rule_cnt+=1
                        
            elif arg2.lower() == 'none':
                for i in range(0,len(head_list)):
                    if (g.lower() in head_list[i] or (g.lower() in body_list[i])):
                        delete_rule.append(i)
                for i in range(0,len(head_list)):
                    if i not in delete_rule:
                        h = head_list[i]
                        b = body_list[i]
                        r = h +' --> '+b
                        rule_list.append(r)
                        rule_cnt+=1
                
            elif arg2 == '1':
                for i in range(0,len(head_list)):
                    if (g.lower() in head_list[i]) or (g.lower() in body_list[i]):
                        rules_ind.append(i)
        if arg2 == '1':
            for r in rules_ind:
                cnt = 0;
                for g in genes:
                    if g.lower() in head_list[r]:
                        cnt +=1
                    elif g.lower() in body_list[r]:
                        cnt +=1
                if cnt <= 1:
                    h = head_list[i]
                    b = body_list[i]
                    r = h +' --> '+b
                    rule_list.append(r)
                    rule_cnt+=1
            
    elif(arg1.lower() == 'head'):
        for g in genes:
            if arg2.lower() == 'any':
                for i in range(0,len(head_list)):
                    if g.lower() in head_list[i]:
                        h = head_list[i]
                        b = body_list[i]
                        r = h +' --> '+b
                        rule_list.append(r)
                        rule_cnt+=1
            elif arg2.lower() == 'none':
                for i in range(0,len(head_list)):
                    if g.lower() in head_list[i]:
                        delete_rule.append(i)
            elif arg2 == '1':
                for i in range(0,len(head_list)):
                    if g.lower() in head_list[i]:
                        rules_ind.append(i)
        if arg2 == '1':
            for r in rules_ind:
                cnt = 0;
                for g in genes:
                    if g.lower() in head_list[r]:
                        cnt +=1
                if cnt <= 1:
                    h = head_list[i]
                    b = body_list[i]
                    r = h +' --> '+b
                    rule_list.append(r)
                    rule_cnt+=1
        if arg2.lower() == 'none':
            for i in range(0,len(head_list)):
                if i not in delete_rule:
                    h = head_list[i]
                    b = body_list[i]
                    r = h +' --> '+b
                    rule_list.append(r)
                    rule_cnt+=1
    elif(arg1.lower() == 'body'):
        for g in genes:
            if arg2.lower() == 'any':
                for i in range(0,len(body_list)):
                    if g.lower() in body_list[i]:
                        h = head_list[i]
                        b = body_list[i]
                        r = h +' --> '+b
                        rule_list.append(r)
                        rule_cnt+=1
            elif arg2.lower() == 'none':
                for i in range(0,len(body_list)):
                    if g.lower() in body_list[i]:
                        delete_rule.append(i)             
            elif arg2 == '1':
                for i in range(0,len(head_list)):
                    if g.lower() in body_list[i]:
                        rules_ind.append(i)
        if arg2 == '1':
            for r in rules_ind:
                cnt = 0;
                for g in genes:
                    if g.lower() in body_list[r]:
                        cnt +=1
                if cnt <= 1:
                    h = head_list[i]
                    b = body_list[i]
                    r = h +' --> '+b
                    rule_list.append(r)
                    rule_cnt+=1
        if arg2.lower() == 'none':
            for i in range(0,len(head_list)):
                if i not in delete_rule:
                    h = head_list[i]
                    b = body_list[i]
                    r = h +' --> '+b
                    rule_list.append(r)
                    rule_cnt+=1
    else:
        print("invalid input.")
    return rule_list, rule_cnt

def template2_results(arg1, arg2, head_list, body_list):
    rule_list = []
    rule_cnt = 0
    if arg1.lower() == 'rule':
        for i in range(0,len(head_list)):
            if (head_list[i].count('g') + body_list[i].count('g')) >= int(arg2):
                h = head_list[i]
                b = body_list[i]
                r = h +' --> '+b
                rule_list.append(r)
                rule_cnt+=1
    elif arg1.lower() == 'head':
        for i in range(0,len(head_list)):
            if (head_list[i].count('g')) >= int(arg2):
                h = head_list[i]
                b = body_list[i]
                r = h +' --> '+b
                rule_list.append(r)
                rule_cnt+=1
    elif arg1.lower() == 'body':
        for i in range(0,len(body_list)):
            if (body_list[i].count('g')) >= int(arg2):
                h = head_list[i]
                b = body_list[i]
                r = h +' --> '+b
                rule_list.append(r)
                rule_cnt+=1
    else:
        print("Invalid input.")
    return rule_list, rule_cnt

def template3_results(operation, left1,left2,left3,right1,right2,right3,head_list,body_list):
    if operation == '1or1' or operation == "1and1":
        op1_list_cnt = template1_results(left1,left2,left3,head_list,body_list)
        op2_list_cnt = template1_results(right1,right2,right3,head_list,body_list)
        if(operation =="1or1"):
            return set(op1_list_cnt[0]).union(set(op2_list_cnt[0]))
        elif(operation=="1and1"):
            return set(op1_list_cnt[0]).intersection(op2_list_cnt[0])
    elif operation == '1or2' or operation == "1and2":
        op1_list_cnt = template1_results(left1,left2,left3,head_list,body_list)
        op2_list_cnt = template2_results(right1,right2,head_list,body_list)
        if(operation =="1or2"):
            return set(op1_list_cnt[0]).union(set(op2_list_cnt[0]))
        elif(operation=="1and2"):
            return set(op1_list_cnt[0]).intersection(op2_list_cnt[0])
    elif operation == '2or2' or operation == "2and2":
        op1_list_cnt = template2_results(left1,left2,head_list,body_list)
        op2_list_cnt = template2_results(right1,right2,head_list,body_list)
        if(operation =="2or2"):
            return set(op1_list_cnt[0]).union(set(op2_list_cnt[0]))
        elif(operation=="2and2"):
            return set(op1_list_cnt[0]).intersection(op2_list_cnt[0])
    else:
        print("invalid operation.")


#Reading data and forming a list of all genes with proper names like G1_UP or G30_DOWN
#Neglecting the disease name
def transform_data(filename):
    gene_data = open(filename)
    gene_seq = gene_data.readlines()
    all_gene_list=[]
    cnt = 0;
    for line in gene_seq:
        genes = line.strip().split('\t')
        gene_set = []
        i=1
        for ele in np.array(genes):
            gene_set.append("g" + str(i) + "_" + ele.lower())
            i = i + 1
            cnt += 1
        all_gene_list.append(gene_set)
    return all_gene_list

#Part2A Forming K itemsize frequent itsemsets.
gene_data_list = transform_data("associationRulesData.txt")
support_percent = int(input('support percentage: '))
sup = int((support_percent/100)*len(gene_data_list))
freq_counts, itemset_list, itemset_map = freq_itemsets(gene_data_list, sup) #computing all frequent item combinations list and map with their sup counts.

#Part2B --> Rule generation and printing rules with the template given.
confidence_percent = int(input('confidence percentage: '))
head_list, body_list = generate_gene_rules(itemset_list, confidence_percent,itemset_map)

template = input('input template type: ')

if(template.lower() == 'template1'):
    arg1 = input('rule1: RULE|HEAD|BODY')
    arg2 = input('rule2: ANY|NUMBER|NONE')
    arg3 = input('list of items , seperated.')
    rules_template1, template1_rule_cnt = template1_results(arg1, arg2, arg3, head_list, body_list)
    print("template1 cnt: ",template1_rule_cnt)
    print(rules_template1)
elif template.lower() == 'template2':
    arg1 = input('rule1: RULE|HEAD|BODY')
    arg2 = input('rule2: number')
    rules_template2, template2_rule_cnt = template2_results(arg1, arg2, head_list, body_list)
    print('template2 count: ', template2_rule_cnt)
    print(rules_template2)
elif template.lower() == 'template3':
    comb = input('input combination 1or1|1and1|1or2|1and2|2or2|2and2')
    if comb == "1or1" or comb == "1and1":
        arg1 = input('op1 rule1: RULE|HEAD|BODY')
        arg2 = input('op1 rule2: ANY|NUMBER|NONE')
        arg3 = input('op1 list of items , seperated.')
        arg4 = input('op2 rule1: RULE|HEAD|BODY')
        arg5 = input('op2 rule2: ANY|NUMBER|NONE')
        arg6 = input('op2 list of items , seperated.')
        rules_template3 = template3_results(comb,arg1, arg2,arg3,arg4,arg5,arg6, head_list, body_list)
    elif comb == "1or2" or comb == "1and2":
        arg1 = input('op1 rule1: RULE|HEAD|BODY')
        arg2 = input('op1 rule2: ANY|NUMBER|NONE')
        arg3 = input('op1 list of items , seperated.')
        arg4 = input('op2 rule1: RULE|HEAD|BODY')
        arg5 = input('op2 rule2: number')
        rules_template3 = template3_results(comb,arg1, arg2,arg3,arg4,arg5,"", head_list, body_list)
    elif comb == "2or2" or comb == "2and2":
        arg1 = input('op1 rule1: RULE|HEAD|BODY')
        arg2 = input('op1 rule2: number')
        arg4 = input('op2 rule1: RULE|HEAD|BODY')
        arg5 = input('op2 rule2: number')
        rules_template3 = template3_results(comb,arg1, arg2,"",arg4,arg5,"", head_list, body_list)
    else:
        input("Invalid combination.")
    print('template3 count: ', len(rules_template3))
    print(rules_template3)
else:
    print("Invalid template type.")

support percentage: 50
{1: 109, 2: 63, 3: 2}
confidence percentage: 70
input template type: template3
input combination 1or1|1and1|1or2|1and2|2or2|2and22and2
op1 rule1: RULE|HEAD|BODYHEAD
op1 rule2: number1
op2 rule1: RULE|HEAD|BODYBODY
op2 rule2: number2
template3 count:  3
{'g72_up --> g59_up,g82_down', 'g82_down --> g59_up,g72_up', 'g96_down --> g59_up,g72_up'}
